In [5]:
import pandas as pd
import requests

In [5]:
df = pd.read_csv("/workspace/arnau/correctness-model-internals/data_for_classification/evaluations/llama3.1_8b_chat/trivia_qa_2_60k/base/main_generations_evaluated.csv")
df['question'] = df['prompt'].str[527:-8]
df.head()

,prompt,answer,ground_truth,correct,question
0,I am going to ask you a question. Answer conci...,Marlene Dietrich,['Florence Marjorie Robertson' 'Anna Neagle' '...,0,Which actress was born Marjorie Robertson in 1...
1,I am going to ask you a question. Answer conci...,Worcester,['Prior of Worcester' 'Worcester Cathedral' 'W...,0,The majority of English Monarchs since the Nor...
2,I am going to ask you a question. Answer conci...,Closed spaces,"['BEDS' ""Captain's bed"" 'California king (bed)...",0,A person who suffers from clinophobia is afrai...
3,I am going to ask you a question. Answer conci...,Dombey and Son,['DOMBEY AND SON' 'Dombey & Son' 'Dombey and s...,1,In which novel by Charles Dickens do the chara...
4,I am going to ask you a question. Answer conci...,James J. Braddock,['James J Braddock' 'JAMES J BRADDOCK' 'James ...,1,Which former World Heavyweight boxing champion...


In [6]:
ids = df['correct'] == 1
df2 = df[ids][1000:4000]
#df2.index = range(0,len(df2))
df2['wrong_answer'] = '-'
df2.head()

,prompt,answer,ground_truth,correct,question,wrong_answer
1462,I am going to ask you a question. Answer conci...,John Everett Millais,['John Millais' 'Sir John Everett Millais' 'Si...,1,Who painted The Boyhood of Raleigh' in 1870?,-
1463,I am going to ask you a question. Answer conci...,Keirin,['KEIRIN' 'Keirin' '競輪' 'Kierin' 'Track cyclin...,1,"What is the name of the track cycling event, o...",-
1464,I am going to ask you a question. Answer conci...,riser,['Risers' 'Riser' 'RISER' 'Riser (disambiguati...,1,"On a staircase, what is the vertical part of t...",-
1467,I am going to ask you a question. Answer conci...,Y.E. Yang,['Y. E. Yang' 'Y E Yang' 'Y.E. Yang' 'Y.E. YAN...,1,By winning the 2009 PGA Championship who becam...,-
1468,I am going to ask you a question. Answer conci...,Elton John,['Sir Elton' 'Rockaday Johnny' 'Elthon John' '...,1,Who wrote the music for 'Billy Elliot'?,-


# Deepseek prompting

In [ ]:
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio  # ✅ Fix for Jupyter notebooks or running loops

# Apply fix
nest_asyncio.apply()

# Sample DataFrame (replace with actual df2)

API_KEY = "api-key"  # Replace with your actual API key
API_URL = "https://api.deepseek.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

async def fetch_answer(session, question):
    """Send a request to DeepSeek API asynchronously."""
    prompt = f"Give a wrong but plausible answer to the following question. Answer as shortly as possible, if possible with only a name or thing. {question}. Remember to give a wrong answer."

    data = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    timeout = aiohttp.ClientTimeout(total=20)
    async with session.post(API_URL, json=data, headers=headers, timeout=timeout) as response:
        if response.status == 200:
            result = await response.json()
            return result['choices'][0]['message']['content']
        else:
            print(f"Error {response.status}: {await response.text()}")
            return "Error"

async def process_data(df):
    """Process API calls in parallel for all rows in df."""
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_answer(session, row['question']) for _, row in df.iterrows()]
        results = await asyncio.gather(*tasks)

        # Assign results back to DataFrame
        df['wrong_answer'] = results

# OpenAI prompting

In [ ]:
import openai
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio  # ✅ Fix for Jupyter notebooks or running loops

# Apply fix
nest_asyncio.apply()

# Sample DataFrame (replace with actual df2)

API_KEY = "api-key"  # Replace with your actual OpenAI API key
API_URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

async def fetch_answer(session, question):
    """Send a request to OpenAI API asynchronously."""
    prompt = f"Give a wrong but plausible answer to the following question. Answer as shortly as possible, if possible with only a name or thing. {question}. Remember to give a wrong answer."

    data = {
        "model": "gpt-4o",  # Use the appropriate OpenAI model
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    timeout = aiohttp.ClientTimeout(total=600)
    async with session.post(API_URL, json=data, headers=headers, timeout=timeout) as response:
        if response.status == 200:
            result = await response.json()
            return result['choices'][0]['message']['content']
        else:
            print(f"Error {response.status}: {await response.text()}")
            return "Error"

async def process_data(df):
    """Process API calls in parallel for all rows in df."""
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_answer(session, row['question']) for _, row in df.iterrows()]
        results = await asyncio.gather(*tasks)
        
        # Assign results back to DataFrame
        df['wrong_answer'] = results
    
    return df

In [12]:
quests1

,question,wrong_answer
1462,Who painted The Boyhood of Raleigh' in 1870?,Claude Monet
1463,"What is the name of the track cycling event, o...",Omnium
1464,"On a staircase, what is the vertical part of t...",Tread
1467,By winning the 2009 PGA Championship who becam...,Tiger Woods
1468,Who wrote the music for 'Billy Elliot'?,Andrew Lloyd Webber.
...,...,...
1603,"Snails, squid, clams, scallops, mussels, etc.,...",Arthropoda.
1604,"Shrimps, barnacles, scorpions, spiders, etc., ...",Mollusca.
1605,In which US state did the Battle of the Little...,Texas
1606,In which US state did the Battle of Yorktown t...,New York


In [18]:
import asyncio

# Split df2 into chunks of 100 rows each
chunk_size = 50
quests_list = [df2[i:i + chunk_size][['question']] for i in range(0, 3000, chunk_size)]

# Run the asyncio loop
async def process_all():
    for quests in quests_list:
        print(f"Running quest: {quests.iloc[0].values[0]}")
        try:
            await process_data(quests)  # Use await instead of asyncio.run
        except Exception as e:
            print(f"Error processing quest: {quests.iloc[0].values[0]} - {e}")
        
        # Wait 1 second before the next batch
        await asyncio.sleep(5)

# Run the main async function
asyncio.run(process_all())

Running quest: Who painted The Boyhood of Raleigh' in 1870?
Running quest: What was the former name of Suriname in South America?
Running quest: Who was crowned by Pope Leo III on Christmas Day 800?
Running quest: What was the real name of Lewis Carroll?
Running quest: Who is the author of the novel 'Girl with a Pearl Earring'?
Running quest: Which great river flows though China, Laos, Cambodia and Vietnam to the South China Sea?
Running quest: In July 1946, which hotel in Jerusalem was bombed by Jewish terrorists because it was the HQ of the British administration?
Running quest: Which company built the USA's first turbojet-powered combat aircraft, the P-80 Shooting Star, which entered service in 1945?
Running quest: In which year was the National Lottery started?
Running quest: The wading birds, curlew and snipe belong to which family of birds?
Running quest: In what year was the 50 pence piece introduced?
Running quest: Who lost his world heavyweight boxing title to Gene Tunney in 1

In [29]:
# Ensure the index alignment before merging
quests = pd.concat(quests_list)
quests = quests.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

# Assign 'wrong_answer' values from `quests` back to `df2`
df2['wrong_answer'] = quests['wrong_answer']


In [30]:
for index, row in df2[:15].iterrows():
    print(f"Question: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Wrong answer: {row['wrong_answer']}")
    print()

Question: Who painted The Boyhood of Raleigh' in 1870?
Answer: John Everett Millais
Wrong answer: Claude Monet.

Question: What is the name of the track cycling event, originating in Japan, in which the racers sprint for victory after having been paced for several laps?
Answer: Keirin
Wrong answer: Tour de France

Question: On a staircase, what is the vertical part of the step called?
Answer: riser
Wrong answer: Baluster.

Question: By winning the 2009 PGA Championship who became the first Asian to win a major golf tournament?
Answer: Y.E. Yang
Wrong answer: Tiger Woods

Question: Who wrote the music for 'Billy Elliot'?
Answer: Elton John
Wrong answer: Andrew Lloyd Webber

Question: Who was the first President of Tanzania? (1961-1985)
Answer: Julius Nyerere
Wrong answer: Julius Caesar

Question: The Turing Test assesses what?
Answer: Artificial intelligence
Wrong answer: The Turing Test assesses cooking skills.

Question: What is the name of the new group, founded in 2010 by Liam Galla

In [31]:
print(len(df2))
# remove '.' from wrong_answer
df2['wrong_answer'] = df2['wrong_answer'].str.replace(r'[.\n*]', '', regex=True)

# drop rows such that wrong_answer is answer
df2_clean = df2[df2['wrong_answer'].str.lower() != df2['answer'].str.lower()]
print(len(df2_clean))
# drop rows such that wrong_answer is contained in answer
df2_clean = df2_clean[~df2_clean.apply(lambda row: row['wrong_answer'].lower() in row['answer'].lower(), axis=1)]
print(len(df2_clean))
# drop rows such that answer is contained in wrong_answer
df2_clean = df2_clean[~df2_clean.apply(lambda row: row['answer'].lower() in row['wrong_answer'].lower(), axis=1)]
print(len(df2_clean))

3000
2938
2935
2912


In [32]:
for index, row in df2_clean[:15].iterrows():
    print(f"Question: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Wrong answer: {row['wrong_answer']}")
    print()

Question: Who painted The Boyhood of Raleigh' in 1870?
Answer: John Everett Millais
Wrong answer: Claude Monet

Question: What is the name of the track cycling event, originating in Japan, in which the racers sprint for victory after having been paced for several laps?
Answer: Keirin
Wrong answer: Tour de France

Question: On a staircase, what is the vertical part of the step called?
Answer: riser
Wrong answer: Baluster

Question: By winning the 2009 PGA Championship who became the first Asian to win a major golf tournament?
Answer: Y.E. Yang
Wrong answer: Tiger Woods

Question: Who wrote the music for 'Billy Elliot'?
Answer: Elton John
Wrong answer: Andrew Lloyd Webber

Question: Who was the first President of Tanzania? (1961-1985)
Answer: Julius Nyerere
Wrong answer: Julius Caesar

Question: The Turing Test assesses what?
Answer: Artificial intelligence
Wrong answer: The Turing Test assesses cooking skills

Question: What is the name of the new group, founded in 2010 by Liam Gallaghe

In [34]:
df2

,prompt,answer,ground_truth,correct,question,wrong_answer
0,I am going to ask you a question. Answer conci...,John Everett Millais,['John Millais' 'Sir John Everett Millais' 'Si...,1,Who painted The Boyhood of Raleigh' in 1870?,Claude Monet
1,I am going to ask you a question. Answer conci...,Keirin,['KEIRIN' 'Keirin' '競輪' 'Kierin' 'Track cyclin...,1,"What is the name of the track cycling event, o...",Tour de France
2,I am going to ask you a question. Answer conci...,riser,['Risers' 'Riser' 'RISER' 'Riser (disambiguati...,1,"On a staircase, what is the vertical part of t...",Baluster
3,I am going to ask you a question. Answer conci...,Y.E. Yang,['Y. E. Yang' 'Y E Yang' 'Y.E. Yang' 'Y.E. YAN...,1,By winning the 2009 PGA Championship who becam...,Tiger Woods
4,I am going to ask you a question. Answer conci...,Elton John,['Sir Elton' 'Rockaday Johnny' 'Elthon John' '...,1,Who wrote the music for 'Billy Elliot'?,Andrew Lloyd Webber
...,...,...,...,...,...,...
2995,I am going to ask you a question. Answer conci...,Constantinople,"['Constantinople, Greece' 'Mickelgard' 'Consta...",1,What Christian city was sacked during the Four...,Vienna
2996,I am going to ask you a question. Answer conci...,William the Conqueror,['King William I of England' 'The Conqueror Wi...,1,Who commissioned the Domesday Book to survey h...,King John
2997,I am going to ask you a question. Answer conci...,Venice,['Sestiere (Venice)' 'Venice' 'Venedig' 'Distr...,1,What city was led by a doge?,Florence
2998,I am going to ask you a question. Answer conci...,Hawaii,['Haiwii' 'The State of Hawaii' 'Mokuʻāina o H...,1,What island chain did Captain Cook discover be...,Galápagos Islands


In [37]:
df2['prompt'] = df2['question'] + ' ' + df2['wrong_answer'] + '.'
df2[['prompt', 'answer']].to_csv('prompts/trivia_qa_2_60k_wrong_answers_1000_4000.csv', index=False)

In [38]:
df2['prompt'] = df2['question'] + ' ' + df2['answer'] + '.'
df2[['prompt', 'answer']].to_csv('prompts/trivia_qa_2_60k_right_answers_1000_4000.csv', index=False)

In [6]:
df_false = pd.read_csv('prompts/trivia_qa_2_60k_wrong_answers_1000_4000.csv')

In [7]:
df_false

,prompt,answer
0,Who painted The Boyhood of Raleigh' in 1870? C...,John Everett Millais
1,"What is the name of the track cycling event, o...",Keirin
2,"On a staircase, what is the vertical part of t...",riser
3,By winning the 2009 PGA Championship who becam...,Y.E. Yang
4,Who wrote the music for 'Billy Elliot'? Andrew...,Elton John
...,...,...
2995,What Christian city was sacked during the Four...,Constantinople
2996,Who commissioned the Domesday Book to survey h...,William the Conqueror
2997,What city was led by a doge? Florence.,Venice
2998,What island chain did Captain Cook discover be...,Hawaii
